# project module 1

![Image](https://retos-operaciones-logistica.eae.es/wp-content/uploads/2020/10/project-charter.jpg)


## The project consists of creating a program in which the user can obtain the closest BiciMad to his place of interest or the closest BiciMad to all his places of interest

<p align="left"><img src="https://www.tuexpertoapps.com/wp-content/uploads/2019/07/bicimad.jpg.webp"></p>


# Step 1: import libraries


![Image](http://www.geomapik.com/wp-content/uploads/2020/05/librerias-python-gis-portada-1024x679-1.jpg)

## before putting my hand to work and importing all the tools with which I am going to work, that is, the libraries
import `pandas` as pd


from `shapely.geometry` import Point

import `geopandas` as gpd

import `argparse`

import `requests`

import `sys`

import `fuzzywuzzy`

from `fuzzywuzzy` import fuzz

from `fuzzywuzzy` import process

# step 2: import data

![image](https://cdn2.excelyvba.com/wp-content/uploads/2015/02/importar-datos-web.jpg)

## In this step I have imported all the data I needed  And the codes and functions used are the following

In this function, what I have done is import the dataset that I have in the data folder that I have previously downloaded from the database that they gave me in the class and I have converted it into a DataFrame

In [7]:
def BiciMad(): 
    df_BiciMad = pd.read_json("data/bicimad_stations.json")
    return df_BiciMad

I have called the API to get my two data files

In [ ]:
def aparcamientos_residentes(): #m_acquisition
    aparcamientos_residentes = requests.get("https://datos.madrid.es/egob/catalogo/202584-0-aparcamientos-residentes.json")
    aparcamientos_residentes1 = aparcamientos_residentes.json()
    df1 = pd.json_normalize(aparcamientos_residentes1['@graph'])
    return df1


def aparcamientos_publicos(): #m_acquisition
    aparcamientos_publicos = requests.get("https://datos.madrid.es/egob/catalogo/202625-0-aparcamientos-publicos.json")
    aparcamientos_publicos1 = aparcamientos_publicos.json()
    df2 = pd.json_normalize(aparcamientos_publicos1['@graph'])
    return df2

# Step 3: Clean Data

![image](https://www.vozidea.com/wp-content/uploads/2015/03/wp-sweep.png)

## The data almost never comes as we would like because in this step we have had to clean the data and keep as useful as possible

To develop my code I needed to have the latitude and longitude separate, one thing that my DataFrame did not offer me because I had to create two columns, one for the latitude and the other for the longitude

In [8]:
def lat_lon(df_BiciMad): 
    lat = [float(column["geometry_coordinates"].split(",")[0].replace("[", "")) for index, column in df_BiciMad.iterrows()]
    lon = [float(column["geometry_coordinates"].split(",")[1].replace("]", "")) for index, column in df_BiciMad.iterrows()]
    df_BiciMad["LATITUD"] = lon
    df_BiciMad["LONGITUD"] = lat
    return df_BiciMad

and also in the DataFrame there were columns that I didn't need and I had to delete them

In [9]:
def clean(df_BiciMad): 
    newdf = df_BiciMad.drop(["activate","no_available","total_bases","dock_bikes","free_bases","reservations_count","geometry_type","geometry_coordinates","light","number","id"], axis='columns')
    newdf['Distance'] = newdf.apply(lambda x: to_mercator(x['LATITUD'], x['LONGITUD']), axis =1)
    return newdf

I have realized that both dataframes have the same columns so we have been able to concatenate the two DataFrame and we have also cleaned and renamed and reordered our DataFrame

In [11]:
def concatenate(df1, df2):
    df_concatenado = pd.concat ([df1, df2] ,ignore_index=True)
    df_clean = df_concatenado.drop(["@id","@type","id","relation","address.district.@id","address.area.@id","address.locality","address.postal-code","organization.accesibility","organization.schedule","organization.organization-desc", "organization.services"], axis='columns')
    df_clean.rename(columns={"title":"Place of interest","organization.organization-name":"Type of place","address.street-address":"Place address","location.latitude":"Latitud", "location.longitude":"Longitud"},inplace=True)
    return df_clean

# Step 4: analyze data

![image](https://opendatacanarias.es/wp-content/uploads/2021/02/analisis-datos.jpg)

## After cleaning the data and having them as we would like, now it is time to analyze them

In [ ]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

in this case we have applied the to_mercator function to obtain the latitude and longitude as requested by the function and we have put it in a new column which we have called `Distance`

In [ ]:
def df_final(df_clean):
    df_clean['Distance'] = df_clean.apply(lambda x: to_mercator(x['Latitud'], x['Longitud']), axis =1)
    df_clean[["Place of interest", "Type of place", "Place address","Latitud","Longitud"]]
    return df_clean

To facilitate the work, it is recommended to have few tables to have a general view of all the data.

In [ ]:
def combine(newdf, df_clean):
    df_combinado = df_clean.merge(newdf, how="cross")
    df_combinado['Distancia'] = df_combinado.apply(lambda x: distance_meters(x['Distance_x'], x['Distance_y']), axis =1)
    df_final = df_combinado.drop(["Latitud","Longitud","Distance_x","LATITUD","LONGITUD","Distance_y"], axis='columns')
    return df_final

# Step 5: mount the program

 ![image](https://miracomosehace.com/wp-content/uploads/2020/06/pantalla-de-pc-table-y-telefono-e-iconos.jpg)

## in this step it is already time to mount our program and facilitate the use to the user

In [2]:
def usuario(df_final):
    for i in range(3):
        a = (input("Si desea obtener la BiciMad más cercana a su lugar de intéres introduce el número 1                                                                                                                                                                  "+
               "Si desea obtener la BiciMad más cercana a todos sus lugares de intéres introduce el número 2                                                                                                                                                         "+"La opcíon intoroducida es la: "))
        if a =="1":
            Lugar_de_interes = str(input("introduce el lugar de interes: "))
            def match_names(Lugar_de_interes, datos_unicos, min_score=0):
                max_score = -1
                max_name = ''
                for x in datos_unicos:
                    score = fuzz.ratio(Lugar_de_interes.lower(), x.lower())
                    if (score > min_score) & (score > max_score):
                        max_name = x
                        max_score = score
                        return max_name
            max_name = match_names(Lugar_de_interes, datos_unicos, min_score=90)
            Pro = df_final[df_final["Type of place"]==max_name]
            busqueda = Pro.sort_values(by="Distancia",ascending=True).head(1)
            busqueda.to_csv("data/BiciMad_Mas_Cercana.csv", sep= ";")
            print("El archivo de la BiciMad más cercana a su lugar de intéres se encuentra guardado en la carpeta data, en formato CSV")
            break
        elif a == "2":
            pro_main = df_final.sort_values(by = "Distancia", ascending = True).groupby('Place of interest')["Type of place",'Place address', 'name','address'].nth(0)
            pro_main.to_csv("data/BiciMads_Mas_Cercana.csv", sep= ";")
            print("El archivo de la BiciMad más cercana a cada uno de sus lugares de intéres se encuentra guardado en la carpeta data, en formato CSV")
            break

        else:
            print("Lo sentimos la opcíon introducida no existe, vuelva a ejecutar de nuevo el programa")

# Step 6: Assemble the pipeline 

![image](https://www.kiuwan.com/wp-content/uploads/2021/08/kiuwan-Increasing-Development-Pipeline-Efficiency.png)

## In this step we have assembled our pipeline including the argparse library creating all the necessary folders and all the necessary `.py ` files for everything to work

In [3]:
from p_acquisition import m_acquisition as mac
from p_wrangling import m_wrangling as mwr
from p_analysis import m_analysis as man 
from p_reporting import m_reporting as mre